In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.07436227798461914
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 11.77605828478104
200000 21.060049072757074
300000 23.652177602759497
400000 27.39403979556954
500000 29.031313646755855
600000 29.366485343138628
700000 28.739202378045952
800000 28.284130268619403
900000 28.15199401615076
1000000 28.088203331959733
1100000 28.236021721969596
1200000 12.20196932183912
1300000 24.051939951947094
1400000 27.82650066169496
1500000 28.008942736988327
1600000 27.92813631358269
1700000 28.163758905528855
1800000 29.82726964055585
1900000 28.22778532956995
2000000 28.02702374895737
2100000 28.27640941860298
2200000 28.46671982337717
2300000 28.659573422730315
2400000 7.426104354601648
2500000 30.17615984568531
2600000 35.5829026549747
2700000 34.62816658047644
2800000 34.27366090785921
2900000 34.51214894403161
3000000 33.211229

29400000 19.508044202715425
29500000 16.74608978418247
29600000 27.229096785641815
29700000 28.147967873941813
29800000 28.042933243999276
29900000 28.926996983372863
30000000 27.91825275268028
30100000 29.084952932259664
30200000 29.48224391913597
30300000 29.12844811613062
30400000 11.948864605234071
30500000 29.147601436657634
30600000 29.06239426483054
30700000 30.617346507640384
30800000 25.43492370935056
30900000 14.751701271223409
31000000 29.28971150217108
31100000 29.3788621371405
31200000 29.490193232893994
31300000 28.42150785731789
31400000 27.837843383427987
31500000 28.148502200311427
31600000 26.980806011972224
31700000 26.688103710063583
31800000 7.985255470134197
31900000 27.15618752194494
32000000 25.327909074367128
32100000 27.543170837097712
32200000 27.2966350668706
32300000 28.666227180614936
32400000 28.151301333306854
32500000 29.78772833147527
32600000 28.559432952157135
32700000 19.41776514584442
32800000 21.374794972636142
32900000 28.82002876648642
33000000 

59200000 26.669566969597255
59300000 11.500874521892397
59400000 24.228332243862507
59500000 25.276545466406223
59600000 25.409516854292516
59700000 23.07281907393259
59800000 23.25604364117871
59900000 22.957702322015123
60000000 13.438388779206534
60100000 17.113857760927395
60200000 23.366122849973483
60300000 23.20743150110483
60400000 23.329311417917726
60500000 24.417878293985655
60600000 24.070879826777052
60700000 20.92281632740649
60800000 16.253887627485767
60900000 26.655489139265175
61000000 26.594880762863454
61100000 25.4641294199062
61200000 26.218128759688195
61300000 26.846073709943575
61400000 26.512699779239348
61500000 26.78101988996499
61600000 25.513249813626445
61700000 9.93946405858739
61800000 25.548595024831922
61900000 25.254880568166936
62000000 24.410892591333823
62100000 23.925310279466366
62200000 23.237361710132785
62300000 24.37685738029567
62400000 5.964885770687526
62500000 24.826195985331854
62600000 23.38784976545136
62700000 23.615376426046964
6280

89000000 22.636403753858733
89100000 22.75062357453116
89200000 23.651073978218218
89300000 22.322321951258886
89400000 23.828185061780374
89500000 24.27544050472341
89600000 23.791740409463575
89700000 24.262878617928862
89800000 23.4417647010162
89900000 23.14034454081846
90000000 22.016231490900154
90100000 12.04632675999686
90200000 17.795967947436154
90300000 20.874987904410062
90400000 22.396153527125573
90500000 21.624823248511223
90600000 9.40477683271867
90700000 19.404976569284543
90800000 21.52885979078838
90900000 21.12586330161708
91000000 22.02946370075522
91100000 13.50058477174459
91200000 21.713989097556755
91300000 22.56878353696903
91400000 22.498648901686593
91500000 23.19465775680257
91600000 23.0981146462263
91700000 23.631677234511006
91800000 24.175308209678622
91900000 23.210179847003197
92000000 23.337730444371832
92100000 23.9514048060121
92200000 23.032171401918017
92300000 22.268708896237943
92400000 21.672803873337248
92500000 7.725391604490529
92600000 21

118100000 19.947727296581643
118200000 20.24141524310751
118300000 19.43890500675094
118400000 19.79470174086905
118500000 17.1261508220617
118600000 9.686871753653884
118700000 20.184182882319014
118800000 13.455229441404144
118900000 11.51011932628596
119000000 19.42553800972802
119100000 12.197826242392424
119200000 15.528298086341405
119300000 19.60486223408655
119400000 18.964154425585424
119500000 20.086423447056866
119600000 22.242615370668187
119700000 20.2964017861237
119800000 20.711314134221865
119900000 21.47862626277304
120000000 20.84984621217545
120100000 21.546925433312076
120200000 21.198694818356344
120300000 20.747452867893752
120400000 19.660861096454894
120500000 19.55229108153872
120600000 19.5242697433715
120700000 19.150735242149423
120800000 19.04180272014749
120900000 12.386994251765397
121000000 13.892610648250422
121100000 18.16218798395338
121200000 5.531069734345174
121300000 19.246489938836227
121400000 16.764966664614715
121500000 10.364284041219554
1216

...

261500000 8.342570498292142
261600000 5.588850387555054
261700000 9.813979735266807
261800000 9.313305294596898
261900000 9.535655661620423
262000000 9.296133770623117
262100000 8.870676932981103
262200000 8.87890677834213
262300000 7.143376172511742
262400000 4.835285415740806
262500000 8.628494813446624
262600000 10.110071859862126
262700000 9.193540551861679
262800000 9.242424030724706
262900000 9.506231571458029
263000000 9.717120797209107
263100000 8.078507970126598
263200000 5.639036477631966
263300000 9.91940875346359
263400000 10.100020989219056
263500000 9.990561874758955
263600000 10.058019197007207
263700000 9.672575008362681
263800000 9.925104594428845
263900000 3.7952945543121492
264000000 10.424400365541722
264100000 9.671190334436018
264200000 9.447886311880255
264300000 9.240472027104586
264400000 9.154099801922076
264500000 8.870164956461744
264600000 8.661625415242941
264700000 3.5150038842611377
264800000 8.229928088564058
264900000 8.856098003625533
265000000 8.8187

290900000 7.4733935755827705
291000000 7.4431202824064915
291100000 7.3205062851866565
291200000 7.499662639948206
291300000 7.5427697280883095
291400000 7.66917798881932
291500000 7.725594147447157
291600000 2.752899149719083
291700000 7.944507192882097
291800000 7.412211024029993
291900000 4.110617535540791
292000000 6.985351496736413
292100000 7.699215699663233
292200000 7.455363336332453
292300000 7.436022174524598
292400000 7.465401025501064
292500000 7.270475643033467
292600000 7.512401744415074
292700000 7.4173579081557515
292800000 7.291101727066892
292900000 4.720516302341696
293000000 4.556741773364984
293100000 7.291835006853295
293200000 7.530998331713525
293300000 7.460909423050671
293400000 7.515517067632917
293500000 7.813265434174913
293600000 7.724257595225838
293700000 7.433280625399506
293800000 7.6230309191602155
293900000 7.023638483492972
294000000 3.435857646847368
294100000 7.342563069308395
294200000 5.8348135442284494
294300000 4.758659806964741
294400000 7.29

319800000 2.4042306696371276
319900000 2.4762642164142807
320000000 2.6502044108132674
320100000 2.4198869961223424
320200000 2.52113310318954
320300000 2.277332441798084
320400000 2.282054672388497
320500000 2.434330717404908
320600000 2.308768478300752
320700000 2.008056802177234
320800000 2.3062970960663
320900000 2.0241531557501844
321000000 1.6716811638166842
321100000 1.2699886972079812
321200000 1.229318516977813
321300000 1.217347828258678
321400000 1.421075298733664
321500000 2.3307892267341694
321600000 2.573394199229875
321700000 2.1203309364296117
321800000 1.9298696316185329
321900000 2.324161128936949
322000000 2.525581157127637
322100000 2.3258472270827513
322200000 2.310587180009628
322300000 2.6612443135417703
322400000 2.3557517388451714
322500000 2.368400076899285
322600000 2.3267043217818477
322700000 2.287934222873643
322800000 2.1137927843161246
322900000 2.0276922328659537
323000000 2.0283284035136497
323100000 2.4276368386215488
323200000 2.1610210255024254
3233

348400000 1.335074114003545
348500000 1.3251745703329498
348600000 1.295475553610946
348700000 1.3734145650875131
348800000 1.1203834421379189
348900000 1.1914061197669445
349000000 1.421225077532343
349100000 1.3585109914357323
349200000 0.973149661208129
349300000 0.6696183543034315
349400000 0.7026826643110528
349500000 0.7157600866459349
349600000 0.9045033949480772
349700000 1.0834640990182505
349800000 1.4019079560143775
349900000 1.1802502082124768
350000000 1.240961615410169
350100000 1.2907096361321793
350200000 1.2138336844807147
350300000 1.2536332370305803
350400000 1.380531899178216
350500000 1.3002583412973185
350600000 1.4562174957429808
350700000 1.3459395646595054
350800000 1.3285377902994693
350900000 1.2184574500931025
351000000 1.2048825921379687
351100000 1.2168050714143812
351200000 1.2274885908737467
351300000 1.1978776678730123
351400000 1.2145180632840078
351500000 1.025815284256124
351600000 0.8497739089665439
351700000 0.7476778132651866
351800000 0.663610128

376200000 0.058755884285799004
376300000 0.05769380229395628
376400000 0.05816449492674801
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.28576402257214917
376600000 0.3538754119143823
376700000 0.34558732000707193
376800000 0.3350294367728214
376900000 0.3086697576482515
377000000 0.30370364735119804
377100000 0.31460697735135756
377200000 0.24326386180752518
377300000 0.24756957568677798
377400000 0.2502380338169072
377500000 0.2520058626921177
377600000 0.2453346241413236
377700000 0.23854056978510285
377800000 0.235735532162257
377900000 0.22790730694678693
378000000 0.21630760026812224
378100000 0.22024815659772753
378200000 0.2282726216470096
378300000 0.22584180289440287
378400000 0.2162849116022905
378500000 0.24694894245824217
378600000 0.25947622983384794
378700000 0.23569612856578032
378800000 0.18287028357337448
378900000 0.04501888110496534
379000000 0.036791747842150925
379100000 0.04078757977888253
379200000 0.04461167128036553
37930000